In [ ]:
import copy
import os
import pandas as pd
import pickle
import shutil
from pathlib import Path

In [ ]:
# Working directories and other cosnsts
work_dir = os.getenv('BASE_WORK_DIR')
base_mpm_dir = f"{work_dir}/quit_mpm_pipeline_output"
scans_dir = f"{work_dir}/scans" # Original scans
state_dir = f"{work_dir}/state"
scripts_dir = f"{work_dir}/scripts"
mt_delta_template_work_dir = f"{work_dir}/magnetisation_transfer_template_workdir"
mni_work_dir = f"{work_dir}/mni_template_workdir"
mni_template_path = f"{mni_work_dir}/tpl-MNIPediatricAsym_cohort-5_res-1_T1w.nii.gz"
randomise_work_dir = f"{work_dir}/fsl_randomise_workdir"
participants_ages_df = pd.read_csv("participants_ages.csv") # Not provided in this git repository
age_bin_names = ["under_and_including_12", "12_to_13", "13_to_14", "14_and_above"]

In [ ]:
# Load available subject_id/session_id state from pickle (not provided in this git repository).
# The available tuples are the one that will go through the MPM pipeline (i.e. only acquistions that include ALL the needed files/modalities will go thorugh the pipeline)
subjects_per_session_pickle_path=f"{state_dir}/subjects_per_session.pickle"
with open(subjects_per_session_pickle_path, 'rb') as handle:
    print("Subject/Session id state was loaded from pickle")
    full_state = pickle.load(handle)
    subjects_per_session = full_state["subjects_per_session"]

# Create a dataframe holding the relevant information needed to continue the analysis
# Map between (subject_id and session_id) to raw scans directory, age, age group, etc.
session_column_name = {1: "Age at T1 scan date (Months)", 2: "Age at T2 scan date (Months)"}
data_list = []

for ses_id, subjects_ids in subjects_per_session.items():
    for subject_id in subjects_ids:
        current_mpm_dir = f"{base_mpm_dir}/sub-{subject_id:03d}/ses-{ses_id:02d}/"
        age = participants_ages_df.loc[participants_ages_df["subject_id"] == f"{subject_id:03d}"][session_column_name[ses_id]].values[0]
        data_list.append({"subject_id": subject_id, "session_id": ses_id, "age": age, "current_mpm_dir": current_mpm_dir})


participant_info_df = pd.DataFrame.from_dict(data_list)
bins = [participant_info_df['age'].min(), 12*12, 13*12, 14*12, participant_info_df['age'].max()]
participant_info_df["age_group_range"] = pd.cut(participant_info_df['age'], bins=bins, include_lowest=True)
participant_info_df["age_group_name"] = pd.cut(participant_info_df['age'], bins=bins, include_lowest=True, labels=age_bin_names)
participant_info_df = participant_info_df.sort_values(['subject_id', 'session_id'], ascending = (True, True))

In [ ]:
# Step 1 - Create the HD-BET masks to be used in the MPM pipeline using QUIT
def create_hd_bet_command_index(dataframe, scans_dir, brain_masks_output_dir, output_index_file, fsl_roi_sh_path)
    textfile = open(output_index_file, "w")
    textfile2 = open(fslroi_file_name, "w")

    for index, row in dataframe.iterrows():
        subject_id = row.subject_id
        ses_id = row.session_id
        age_group_name = row.age_group_name
        current_t1_mpm_file = f"{scans_dir}/sub-{subject_id:03d}/ses-{ses_id:02d}/MPM_T1w.nii.gz"
        dst_file = f"{brain_masks_dir}/sub-{subject_id:03d}_ses-{ses_id:02d}_MPM_T1w.nii.gz"
        if not os.path.isfile(current_t1_mpm_file):
            print(f"{current_t1_mpm_file} doesn't exist")
        
        if not os.path.isfile(dst_file):
            print(f"Copying {current_t1_mpm_file} to {dst_file}")
            shutil.copy(current_t1_mpm_file, dst_file)
        else:
            print(f"Dst file {dst_file} already exists")
        
        # Extract first volume from the MPM T1w image and run HD-BET on that volume
        vol0 = f"{brain_masks_dir}/vol_zero/sub-{subject_id:03d}_ses-{ses_id:02d}_MPM_T1w.nii.gz"
        textfile2.write(f"fslroi {dst_file} {vol0} 0 1\n")
        textfile.write(f"python hd-bet -device cpu -tta 0 -i {vol0} -o {brain_masks_dir}/output/sub-{subject_id:03d}_ses-{ses_id:02d}_T1_hd_bet_mask.nii.gz\n")
        
    textfile2.close()
    textfile.close()

create_hd_bet_command_index(participant_info_df, scans_dir, f"{base_mpm_dir}/brain_masks_work_dir", f"{scripts_dir}/brain_mask_creation.index", f"{scripts_dir}/fsl_roi.sh")

# Then, run fsl_roi.sh
# Create an SGE job file that receives the index file and and index, and run it as following:
# qsub -t 1:[N] brain_mask_creation.job, where N is the number of rows in the brain_mask_creation.index file
# Then, run the QUIT MPM pipeline

In [ ]:
# Step 2 - After the QUIT MPM pipeline finishes running, copy all the original MTsat files to the mt_delta_template_work_dir
def copy_mt_delta_files_to_template_work_dir(age_group_name):
    for index, row in participant_info_df.loc[participant_info_df["age_group_name"] == age_group_name].iterrows():
        subject_id = row.subject_id
        ses_id = row.session_id
        current_mt_delta_filename = f"{base_mpm_dir}/sub-{subject_id:03d}/ses-{ses_id:02d}/MTSat_delta.nii.gz"

        dest_folder = f"{mt_delta_template_work_dir}/{age_group_name}"
        if not os.path.exists(dest_folder):
            os.mkdir(dest_folder)

        copy_to_path = f"{dest_folder}/sub-{subject_id:03d}_ses-{ses_id:02d}_MTSat_original_space.nii.gz"
        if Path(copy_to_path).is_file():
            print(f"{copy_to_path} already exists, skipping")
        else:
            print(f"Copying MT_Delta file {current_mt_delta_filename} to {copy_to_path}")
            copyfile(current_mt_delta_filename, copy_to_path)

for age_group_name_in age_bin_names:
    copy_mt_delta_files_to_template_work_dir(age_group_name)

# Step 3 - run a manual quality control (QC) step and create an "excluded" directory under each age_group specific directory.
#   Move the images that are not suitable for template creation into the /excluded directory
#
# Step 4 - now, we want to create the MTsat templates per age group.
#   1. Copy the ANTs antsMultivariateTemplateConstruction2.sh scripts into each one of the age_group directories under mt_delta_template_work_dir
#   2. Edit it to your liking according to your wanted SGE runtime configuration (e.g. QSUBOPTS)
#   3. Run: e.g. ./antsMultivariateTemplateConstruction2.sh -b 0 -c 1 -d 3 -i 4 -g 0.2 -k 1 -n 0 -r 1 -o ${PWD}/mt_delta_template_14_and_above_output_ ${PWD}/sub*.nii.gz

In [10]:
# Step 5 - registering the remaining excluded MTsat images into the resulting age-appropriate MTsat template
#   For each age group, run the register_mt_to_{age_group_name}_template.sh file
def create_mt_to_template_syn_registrations_sh_file(mt_template_work_dir, age_group_name):
    mt_to_template_work_dir = Path(f"{mt_template_work_dir}/{age_group_name}/excluded")
    shell_output_path = mt_to_template_work_dir / f"register_mt_to_{age_group_name}_template.sh"
    with open(shell_output_path.as_posix(), 'w') as handle:
        for current_mt_file in mt_to_template_work_dir.glob("sub*_ses*_MTSat_original_space.nii.gz"):
            clean_file_name = current_mt_file.stem.split(".")[0]
            reg_cmd = f"cd {mt_to_template_work_dir.as_posix()}; antsRegistrationSyN.sh -d 3 -f ../mt_delta_template_{age_group_name}_output_template0.nii.gz -m {current_mt_file.as_posix()} -o mt_delta_template_{age_group_name}_output_template0{clean_file_name} -n 4 -t s"
            mv_cmd = f"mv mt_delta_template_{age_group_name}_output_template0{clean_file_name}Warped.nii.gz mt_delta_template_{age_group_name}_output_template0{clean_file_name}99WarpedToTemplate.nii.gz\n"
            handle.write(f"{reg_cmd}; {mv_cmd}")


for age_group_name in age_bin_names:
    create_mt_to_template_syn_registrations_sh_file(mt_delta_template_work_dir, age_group_name)

In [ ]:
# Step 6 - copying the resulting transformation files for each MTsat image in the age-appropriate MTsat template to the relevant MPM folder
#   Two transformation files will be copied - the SyN deformation file (*1Warp.nii.gz file) and the affine transformation file (0GenericAffine.mat)
#   Last, the MTsat images transformed into the MTsat template space are also copied.
def copy_mt_to_mt_template_transformations_to_mpm_dir(mt_template_work_dir, age_group_name, dst_mpm_dir):
    def mt_inner_copy(current_dir):
        for affine_file in sorted(current_dir.glob("*0GenericAffine.mat")):
            p = affine_file.as_posix()
            if p.endswith("template0GenericAffine.mat"):
                print(f"Skipping template file {p}")
                continue
            groups = re.search(r"sub-([0-9]{3})_ses-([0-9]{2})", p).groups()
            copy_to_path = f"{dst_mpm_dir}/sub-{groups[0]}/ses-{groups[1]}/Reg_MT_To_{age_group_name}_MT_Template_0GenericAffine.mat"
            
            if Path(copy_to_path).is_file():
                print(f"{copy_to_path} already exists, skipping")
            else:
                print(f"Copying Generic Affine file {p} to {copy_to_path}")
                shutil.copyfile(p, copy_to_path)

        print("\n")
        for syn_deformation_file in sorted(current_dir.glob("*1Warp.nii.gz")):
            p = syn_deformation_file.as_posix()
            groups = re.search(r"sub-([0-9]{3})_ses-([0-9]{2})", p).groups()
            copy_to_path = f"{dst_mpm_dir}/sub-{groups[0]}/ses-{groups[1]}/Reg_MT_To_{age_group_name}_MT_Template_1Warp.nii.gz"
            
            if Path(copy_to_path).is_file():
                print(f"{copy_to_path} already exists, skipping")
            else:
                print(f"Copying SyN deformation file {p} to {copy_to_path}")
                shutil.copyfile(p, copy_to_path)

        print("\n")
        for mt_warped_to_template_file in sorted(current_dir.glob("*WarpedToTemplate.nii.gz")):
            p = mt_warped_to_template_file.as_posix()
            groups = re.search(r"sub-([0-9]{3})_ses-([0-9]{2})", p).groups()
            dst_dir = f"{dst_mpm_dir}/sub-{groups[0]}/ses-{groups[1]}/mpm_in_mt_template_space"
            
            if not os.path.isdir(dst_dir):
                os.mkdir(dst_dir)
            
            copy_to_path = f"{dst_dir}/MTSat_delta_sub-{groups[0]}_ses-{groups[1]}_MT_template_space.nii.gz"
            
            if Path(copy_to_path).is_file():
                print(f"{copy_to_path} already exists, skipping")
            else:
                print(f"Copying MT file warped into MT template space {p} to {copy_to_path}")
                shutil.copyfile(p, copy_to_path)
        print("\n")

    age_group_template_work_dir = Path(f"{mt_template_work_dir}/{age_group_name}")
    mt_inner_copy(age_group_template_work_dir)
    mt_inner_copy(age_group_template_work_dir / "excluded")

for age_group_name in age_bin_names:
    copy_mt_to_mt_template_transformations_to_mpm_dir(mt_delta_template_work_dir, age_group_name, base_mpm_dir)

In [ ]:
# Step 7 - Creating the index file to transform all the MPM images (R1, R2*, PD) from their original space into the age-appropriate MTsat template space
def create_mpm_to_mt_transform_command_index(mpm_dir, mt_age_group_templates_work_dir, dataframe, mpm_to_mt_template_index_filename):
    textfile = open(mpm_to_mt_template_index_filename, "w")
        
    for index, row in dataframe.iterrows():
        subject_id = row.subject_id
        ses_id = row.session_id
        age_group_name = row.age_group_name
        current_dir = f"{mpm_dir}/sub-{subject_id:03d}/ses-{ses_id:02d}"
        mt_age_group_template_path = f"{mt_age_group_templates_work_dir}/{age_group_name}/mt_delta_template_{age_group_name}_output_template0.nii.gz"

        # MTsat in MT template space already exists - skip it.
        # The following don't exist - need to create
        r1_original_space = f"{current_dir}/MTSat_R1.nii.gz"
        pd_original_space = f"{current_dir}/MTSat_PD.nii.gz"
        r2s_original_space = f"{current_dir}/MPM_R2s.nii.gz"

        syn_transformation_path = f"{current_dir}/Reg_MT_To_{age_group_name}_MT_Template_1Warp.nii.gz"
        affine_transformation_path = f"{current_dir}/Reg_MT_To_{age_group_name}_MT_Template_0GenericAffine.mat"

        result_dir = f"{current_dir}/mpm_in_mt_template_space"
        if not os.path.isdir(result_dir):
            os.mkdir(result_dir)

        # Don't exist - need to create
        r1_in_mt_path = f"{result_dir}/MTSat_R1_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"
        pd_in_mt_path = f"{result_dir}/MTSat_PD_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"
        r2s_in_mt_path = f"{result_dir}/MPM_R2s_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"

        files_to_process = [(r1_original_space, r1_in_mt_path),
        (pd_original_space, pd_in_mt_path),
        (r2s_original_space, r2s_in_mt_path)]

        for input_file, output_file in files_to_process:
            command_to_run = f"cd {current_dir}; antsApplyTransforms -d 3 -i {input_file} -r {mt_age_group_template_path} -t {syn_transformation_path} -t {affine_transformation_path} -o {output_file}"
            textfile.write(command_to_run + "\n")
            print(command_to_run)
        print("\n")
        
    textfile.close()

output_filename = f"{scripts_dir}/mpm_to_mt_template.index"
create_mpm_to_mt_transform_command_index(base_mpm_dir, mt_delta_template_work_dir, participant_info_df, output_filename)

# Step 8 - now, we want to transform the R1, R2* and PD images from their original space into the age-appropriate MTsat template space
# Create an SGE job file that receives the index file and and index, and run it as following:
# qsub -t 1:[N] mpm_to_mt_template.job, where N is the number of rows in the mpm_to_mt_template.index file

In [ ]:
# Step 9 - Creating the index file to transform all the MPM images (R1, R2*, PD) from the age-appropriate MTsat template space into the MNI template space
# 1. Manually register the age-appropriate MTsat templates into the MNI template using antsRegistrationSyN.sh.
# 2. Copy the resulting transformation files to the mt_age_group_templates_work_dir/{age_group_name}/mni_registration directory for each age group.
# 3. Run the fuction below
def create_mpm_in_mt_to_mni_transform_command_index(mpm_dir, mt_age_group_templates_work_dir, mni_template_path, dataframe, mpm_in_mt_to_mni_index_filename):
    textfile = open(mpm_in_mt_to_mni_index_filename, "w")
        
    for index, row in dataframe.iterrows():
        subject_id = row.subject_id
        ses_id = row.session_id
        age_group_name = row.age_group_name
        current_dir = f"{mpm_dir}/sub-{subject_id:03d}/ses-{ses_id:02d}"

        mt_delta_in_mt_path = f"{current_dir}/mpm_in_mt_template_space/MTSat_delta_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"
        r1_in_mt_path = f"{current_dir}/mpm_in_mt_template_space/MTSat_R1_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"
        pd_in_mt_path = f"{current_dir}/mpm_in_mt_template_space/MTSat_PD_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"
        r2s_in_mt_path = f"{current_dir}/mpm_in_mt_template_space/MPM_R2s_sub-{subject_id:03d}_ses-{ses_id:02d}_MT_template_space.nii.gz"

        mt_age_group_template_path = f"{mt_age_group_templates_work_dir}/{age_group_name}/mni_registration"
        syn_transformation_path = f"{mt_age_group_template_path}/mt_delta_template_{age_group_name}_mni_space_1Warp.nii.gz"
        affine_transformation_path = f"{mt_age_group_template_path}/mt_delta_template_{age_group_name}_mni_space_0GenericAffine.mat"

        result_dir = f"{current_dir}/mpm_mt_to_mni"
        if not os.path.isdir(result_dir):
            os.mkdir(result_dir)
        
        mt_delta_in_mni_path = f"{result_dir}/MTSat_delta_sub-{subject_id:03d}_ses-{ses_id:02d}_MNI_space.nii.gz"
        r1_in_mni_path = f"{result_dir}/MTSat_R1_sub-{subject_id:03d}_ses-{ses_id:02d}_MNI_space.nii.gz"
        pd_in_mni_path = f"{result_dir}/MTSat_PD_sub-{subject_id:03d}_ses-{ses_id:02d}_MNI_space.nii.gz"
        r2s_in_mni_path = f"{result_dir}/MPM_R2s_sub-{subject_id:03d}_ses-{ses_id:02d}_MNI_space.nii.gz"

        files_to_process = [(mt_delta_in_mt_path, mt_delta_in_mni_path),
        (r1_in_mt_path, r1_in_mni_path),
        (pd_in_mt_path, pd_in_mni_path),
        (r2s_in_mt_path, r2s_in_mni_path)]

        for input_file, output_file in files_to_process:
            command_to_run = f"cd {current_dir}; antsApplyTransforms -d 3 -i {input_file} -r {mni_template_path} -t {syn_transformation_path} -t {affine_transformation_path} -o {output_file}"
            textfile.write(command_to_run + "\n")
            print(command_to_run)
        print("\n")
        
    textfile.close()

output_filename = f"{scripts_dir}/mpm_in_mt_to_mni.index"
create_mpm_in_mt_to_mni_transform_command_index(base_mpm_dir, mt_delta_template_work_dir, mni_template_path, participant_info_df, output_filename)

# Step 10 - now, we want to transform the R1, R2* and PD images from the age-appropriate MTsat template space into the MNI template space
# Create an SGE job file that receives the index file and and index, and run it as following:
# qsub -t 1:[N] mpm_in_mt_to_mni.job, where N is the number of rows in the mpm_in_mt_to_mni.index file

In [ ]:
# This dataframe holds a subject_id, session_id and age_group_name of the excluded acquisition (not provided in this git repository)
excluded_from_randomise_df = pd.DataFrame.from_csv(f"{state_dir}/excluded_acquisitions.csv")
excluded_from_randomise_df['tuple_col'] = list(zip(excluded_from_randomise_df.subject_id, excluded_from_randomise_df.session_id))

In [ ]:
# Step 11 - merging all MTsat images in the MNI space for a 2nd round of quality control (following registration), before any statistical analysis takes place
# Create an "fsl merge" command to concat all (before any quality control) MTsat MPM files into a single 4d nii.gz file
def create_mtsat_images_fsl_merge_command(mpm_work_dir, df, output_path):
      with open(f"{output_path}/merge_mt_all_pre_qc_sorted.sh", "w") as handle:
            mt_delta_images_all = [f"{mpm_work_dir}/sub-{row.subject_id:03d}/ses-{row.session_id:02d}/mpm_mt_to_mni/MTSat_delta_sub-{row.subject_id:03d}_ses-{row.session_id:02d}_MNI_space.nii.gz" for index,row in df.iterrows()]
            handle.write("fslmerge -t 4d_all_mt_pre_qc_sorted.nii.gz " + " ".join(mt_delta_images_all) + "\n")

create_mtsat_images_fsl_merge_command(base_mpm_dir, participant_info_df.sort_values("age"), randomise_work_dir)

In [ ]:
# Step 12 - merging all MTsat, R1, R2*, PD images that passed QC in order to perform statistical analysis using fsl randomise on each 4d file
def merge_per_modality_after_qc_fsl_merge_command(mpm_work_dir, df, excluded_images_df, output_path):
      excluded_list = list(excluded_images_df.tuple_col.values)
      with open(f"{output_path}/merge_mt_all_sorted.sh", "w") as handle:
            mt_delta_images_all = [f"{mpm_work_dir}/sub-{row.subject_id:03d}/ses-{row.session_id:02d}/mpm_mt_to_mni/MTSat_delta_sub-{row.subject_id:03d}_ses-{row.session_id:02d}_MNI_space.nii.gz" for index,row in df.iterrows() if (row.subject_id, row.session_id) not in excluded_list]
            handle.write("fslmerge -t 4d_all_mt_sorted.nii.gz " + " ".join(mt_delta_images_all) + "\n")
      with open(f"{output_path}/merge_r1_all_sorted.sh", "w") as handle:
            r1_images_all = [f"{mpm_work_dir}/sub-{row.subject_id:03d}/ses-{row.session_id:02d}/mpm_mt_to_mni/MTSat_R1_sub-{row.subject_id:03d}_ses-{row.session_id:02d}_MNI_space.nii.gz" for index,row in df.iterrows() if (row.subject_id, row.session_id) not in excluded_list]
            handle.write("fslmerge -t 4d_all_r1_sorted.nii.gz " + " ".join(r1_images_all) + "\n")
      with open(f"{output_path}/merge_r2s_all_sorted.sh", "w") as handle:
            r2s_images_all = [f"{mpm_work_dir}/sub-{row.subject_id:03d}/ses-{row.session_id:02d}/mpm_mt_to_mni/MPM_R2s_sub-{row.subject_id:03d}_ses-{row.session_id:02d}_MNI_space.nii.gz" for index,row in df.iterrows() if (row.subject_id, row.session_id) not in excluded_list]
            handle.write("fslmerge -t 4d_all_r2s_sorted.nii.gz " + " ".join(r2s_images_all) + "\n")


merge_per_modality_after_qc_fsl_merge_command(base_mpm_dir, participant_info_df.sort_values("age"), excluded_from_randomise_df, randomise_work_dir)

In [ ]:
# Step 13 - Statistical analysis:
# FSL randomise:
#   1. Create design and matrix files (age_corr_sorted.mat and age_corr.con).
#   2. Run:
#       randomise_parallel -i 4d_all_mt_sorted.nii.gz -o mt_all_age_corr_sorted_qc2 -d age_corr_sorted.mat -t age_corr.con -m wm_gm_mask.nii.gz -n 10000 -T -D
#       randomise_parallel -i 4d_all_r1_sorted.nii.gz -o r1_all_age_corr_sorted_qc2 -d age_corr_sorted.mat -t age_corr.con -m wm_gm_mask.nii.gz -n 10000 -T -D
#       randomise_parallel -i 4d_all_r2s_sorted.nii.gz -o r2s_all_age_corr_sorted_qc2 -d age_corr_sorted.mat -t age_corr.con -m wm_gm_mask.nii.gz -n 10000 -T -D
#   3. Threshold the resulting images with values above 0.95 for statistical significance
#
# ROI analysis:
#   1. Create a binary mask for each region. For example, in the Harvard-Oxford Subcortical Atlas, the left pallidum has a value of "7".
#       e.g. fslmaths HarvardOxford-sub-maxprob-thr0-1mm.nii.gz -thr 7 -uthr 7 -bin l_pallidum.nii.gz
#   2. For each permutation of (modality, ROI) run fslstats to output the mean value.
#       e.g. fslstats -t 4d_all_mt_sorted.nii.gz -k l_pallidum.nii.gz -m
#   3. Analyse mean MTsat, R1 and R2* values in each ROI using the output of fslstats.